In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import dicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt

from glob import glob
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import itertools
from IPython.display import FileLink

from sklearn.metrics import log_loss

In [ ]:
# IMAGES_DIR = 'data/sample_images/'
# patient_ids = os.listdir(IMAGES_DIR)
# patient_ids = [f for f in patient_ids if len(f) == 32]

In [ ]:
images_dir = 'data/images/stage1/'
labels = pd.read_csv('data/stage1_labels.csv')

In [ ]:
def load_scan(patient_id):
    dicom_files = glob(os.path.join(images_dir, patient_id, '*.dcm'))
    slices = [dicom.read_file(f) for f in dicom_files]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    return slices

def rescale(slices):
    # normalise pixel array values (HU units?)
    image = np.stack([s.pixel_array for s in slices])
    image = image.astype(np.int16)
    image[image == -2000] = 0 # 0 + intercept = 0 + -1024 = -1024 (air)
    for slice_idx in range(len(slices)):
        image[slice_idx] *= np.int16(slices[slice_idx].RescaleSlope) # slope
        image[slice_idx] += np.int16(slices[slice_idx].RescaleIntercept) # intercept
    return image

def resample(image, slices):
    # make distance between pixels 1mm
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)

    for s in slices:
        s.SliceThickness = slice_thickness
    
    current_shape = np.array([slices[0].SliceThickness] + slices[0].PixelSpacing, dtype=np.float32)
    new_image = scipy.ndimage.interpolation.zoom(image, current_shape, mode='nearest', order=0)
    
    return new_image

def normalise(image):
    minimum = np.min(image)
    maximum = np.max(image)
    new_image = (image - minimum) / (maximum - minimum)
    # ni -= 0.25 # global pixel mean
    return new_image

def preprocess(patient_id):
    slices = load_scan(patient_id)
    original_image = rescale(slices) # 3D image
    new_image = resample(original_image, slices)
    new_image = normalise(new_image)
    new_image = np.swapaxes(new_image, 0, 2)
    new_image = np.reshape(new_image, new_image.shape + (1,))
    return new_image

def get_patient_label(patient_id):
    return labels[labels['id'] == patient_id]['cancer']

## Organise data into train, valid and test dirs

In [ ]:
# split ids into train and valid
valid_percent = 0.15
pos = labels[labels['cancer'] == 1]
neg = labels[labels['cancer'] == 0]
pos_valid_ids = set(pos.sample(frac=valid_percent)['id'])
neg_valid_ids = set(neg.sample(frac=valid_percent)['id'])
pos_train_ids = set(pos['id']).difference(pos_valid_ids)
neg_train_ids = set(neg['id']).difference(neg_valid_ids)
# len(pos_train_ids), len(neg_train_ids), len(pos_valid_ids), len(neg_valid_ids)

pos_train_ids = list(pos_train_ids)
neg_train_ids = list(neg_train_ids)
pos_valid_ids = list(pos_valid_ids)
neg_valid_ids = list(neg_valid_ids)
np.random.shuffle(pos_train_ids)
np.random.shuffle(neg_train_ids)
np.random.shuffle(pos_valid_ids)
np.random.shuffle(neg_valid_ids)

In [ ]:
# def gen_data(patient_ids):
#     for patient_id in iter(patient_ids):
#         yield preprocess(patient_id)

In [ ]:
def gen_data(patient_ids):
    while 1:
        for i in range(10): # 10 * 32 = 320 -> # of training samples
            print("i = " + str(i))
            ids = patient_ids[i*32:(i+1)*32]
            X = [preprocess(patient_id) for patient_id in ids]
            y = [get_patient_label for patient_id in ids]
            yield X, y

In [ ]:
# X_train = gen_data(pos_train_ids + neg_train_ids)
# y_train = np.vstack((np.ones(len(pos_train_ids)).reshape(-1, 1), np.zeros(len(neg_train_ids)).reshape(-1, 1)))

### Define Model

In [ ]:
# from theano.sandbox import cuda

from tensorflow.python.client import device_lib
import tensorflow as tf

from keras.models import Model
from keras import layers
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Conv3D
from keras.layers import MaxPooling3D
from keras.layers import AveragePooling3D
from keras.layers import GlobalAveragePooling3D
from keras.layers import GlobalMaxPooling3D
from keras.engine.topology import get_source_inputs
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.preprocessing import image as k_image

import pandas as pd
import numpy as np

print(K.image_data_format()) # channels last
print(device_lib.list_local_devices())

In [ ]:
# https://github.com/fchollet/deep-learning-models/blob/master/inception_v3.py
channel_axis = 4

def conv_block(x, filters, h, w, z, padding='same', strides=(1, 1, 1)):
    x = Conv3D(filters,
               (h, w, z), # height, width, z
               strides=strides,
               padding=padding,
               use_bias=False)(x)
    x = BatchNormalization(axis=channel_axis, scale=False)(x) # batch norm axis=4 ??
    x = Activation('relu')(x)
    return x

def inception_block(x):
    
    x = conv_block(x, 32, 3, 3, 3, strides=(2, 2, 2), padding='valid')
    x = conv_block(x, 32, 3, 3, 3, padding='valid') # padding same or valid ??
    x = conv_block(x, 64, 3, 3, 3)
    x = MaxPooling3D((3, 3, 3), strides=(2, 2, 2))(x)

    x = conv_block(x, 80, 1, 1, 1, padding='valid')
    x = conv_block(x, 192, 3, 3, 3, padding='valid')
    x = MaxPooling3D((3, 3, 3), strides=(2, 2, 2))(x)

    branch1 = conv_block(x, 64, 1, 1, 1)

    branch5 = conv_block(x, 48, 1, 1, 1)
    branch5 = conv_block(branch5, 64, 5, 5, 5)

    branch3 = conv_block(x, 64, 1, 1, 1)
    branch3 = conv_block(branch3, 96, 3, 3, 3)
    branch3 = conv_block(branch3, 96, 3, 3, 3)

    branch_pool = AveragePooling3D((3, 3, 3), strides=(1, 1, 1), padding='same')(x)
    branch_pool = conv_block(branch_pool, 32, 1, 1, 1)
    
    x = layers.concatenate(
            [branch1, branch5, branch3, branch_pool],
            axis=channel_axis)
    return x

In [ ]:
tf.reset_default_graph()

In [ ]:
with tf.device('/gpu:0'):
    model_input = Input(shape=(None, None, None, 1)) # (x, y, z, channels)
    x = inception_block(model_input)
    x = GlobalAveragePooling3D(name='avg_pool')(x)
    model_output = Dense(1, activation='softmax', name='predictions')(x)
    model = Model(model_input, model_output, name='moses')
    model.compile(optimizer='rmsprop', # change to Adam() or even Eve() ??
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.summary(line_length=125)

## Train

In [ ]:
X = np.expand_dims(preprocess(pos_train_ids[0]), axis=0)

In [ ]:
trn = np.vstack((X,X))

In [ ]:
model.fit(trn, np.array([[1], [1]]))

In [ ]:
# model.fit_generator(gen_data(pos_train_ids + neg_train_ids), steps_per_epoch=100, epochs=1)